# Lesson 2
In this document we will improve on the scripts we created earlier such that the creation of a node and its respective lease and public IP are taken care of.

And do other interesting things.

## Automating creation of the node

In [ ]:
#! /bin/bash

#
LEASE_NAME=$USER"-haswell-test"
SERVER_NAME=$USER"-first-server"
NETWORK_NAME="sharednet1"
EXTERNAL_NETWORK_NAME="public"

# Set up user's project (Replace with your project name)
# export OS_PROJECT_NAME='my-first-project'
# Set region (Optional, default to 'CHI@UC')"
# export OS_REGION_NAME='CHI@UC'

# Functions

CreateLease()
{
   local lease_name="$1"

   blazar lease-create --physical-reservation min=2,max=5,resource_properties='["=", "$node_type", "compute_haswell"]' $lea
se_name
}

# Should make it verify all the arguments are valid before doing anything
CreateServer()
{
   local server_name="$1"
   local server_type="$2"
   local image_name="$3"
   local nic_ID="$4"
   local lease_ID="$5"

   openstack server create \
      --flavor "$server_type" \
      --image "$image_name" \
      --nic net-id="$nic_ID" \
      --hint reservation="$lease_id" \
      "$server_name"
}

# Create lease as needed
lease_exists=$(blazar lease-list --format value -c name 2>/dev/null  | grep "$LEASE_NAME")

if [ ! -z $lease_exists ]; then
   echo -n "Lease $LEASE_NAME already exists... "
   lease_status=$(blazar lease-show --format value -c status "$LEASE_NAME")

   # Lease in a sorry state
   if [[ $lease_status == "TERMIN"* ]] || [[ $lease_status == "ERROR" ]] ; then
      echo "but it is in a sorry state. Restarting it."
      # Delete old lease
      blazar lease-delete "$LEASE_NAME"
      # Create lease (instance reservation) named $LEASE_NAME
      # blazar lease-create --physical-reservation min=2,max=5,resource_properties='["=", "$node_type", "compute_haswell"]'
 $LEASE_NAME
      CreateLease $LEASE_NAME
   else
      echo "and is usable. So we will use it."
   fi
else
   echo "Lease $LEASE_NAME does not exist, so we will create it."
   # Create lease (instance reservation) named $LEASE_NAME
   # blazar lease-create --physical-reservation min=1,max=1,resource_properties='["=", "$node_type", "compute_haswell"]' $L
EASE_NAME
   CreateLease $LEASE_NAME
fi

# Wait until lease is ready before continuing
# Another way is to keep track of PID and wait until it is done
while [[ $lease_status != "ACTIVE" ]]
do
   sleep 5
   lease_status=$(blazar lease-show --format value -c status "$LEASE_NAME")
done

echo "Lease $LEASE_NAME ready for business"

# Launch server SERVER_NAME on network NETWORK_NAME on lease LEASE_NAME

# First, is there a server called SERVER_NAME in use? server_status="" if the answer is no
server_status=$(openstack server list --format value -c Status --name "$SERVER_NAME")

# We need to get the network ID
network_id=$(openstack network show --format value -c id "$NETWORK_NAME")
# We better also know which lease we are going to run this on
lease_id=$(blazar lease-show  --format value -c  reservations "$LEASE_NAME" |grep \"id\"| cut -d \" -f4)

# Is there a server running already?
if [ ! -z $server_status ]
then
   echo -n "Server $SERVER_NAME already exists... "

   if [[ $server_status == "ERROR" ]]
   then
      echo "but it is in a sorry state. Restarting it."
      # Kill broken server
      openstack server delete $SERVER_NAME

      # Now we can create the server SERVER_NAME
      CreateServer "$SERVER_NAME" baremetal CC-CentOS7 "$network_id" "$lease_id"
   else
      echo "and is usable. So we will use it."
   fi
else
   echo "Server $SERVER_NAME does not exist, so we will create it."
   CreateServer "$SERVER_NAME" baremetal CC-CentOS7 "$network_id" "$lease_id"
   # openstack server create --flavor baremetal --image CC-CentOS7 --nic net-id="$network_id" \
        # --hint reservation="$lease_id" "$SERVER_NAME"
fi

# Now give a public IP to server if it is up and running

# Once again we need to see if server is up and running. No point in wasting time on
# a missing or boink server.
# server_status=$(openstack server list --format value -c Status --name "$SERVER_NAME")
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   server_status=$(openstack server list --format value -c Status --name "$SERVER_NAME")
done

if [ ! -z $server_status ] && [[ $server_status != "ERROR" ]]
then
   echo -n "$SERVER_NAME seems to be up and running. So we will add an external IP: "
   # Request a public floating IP (in the EXTERNAL_NETWORK_NAME network)
   if [[ $server_status == "ERROR" ]]
   then
      echo "but it is in a sorry state. Restarting it."
      # Kill broken server
      openstack server delete $SERVER_NAME

      # Now we can create the server SERVER_NAME
      CreateServer "$SERVER_NAME" baremetal CC-CentOS7 "$network_id" "$lease_id"
   else
      echo "and is usable. So we will use it."
   fi
else
   echo "Server $SERVER_NAME does not exist, so we will create it."
   CreateServer "$SERVER_NAME" baremetal CC-CentOS7 "$network_id" "$lease_id"
fi

# Now give a public IP to server if it is up and running
# Once again we need to see if server is up and running. No point in wasting time on
# a missing or boink server.
while [[ $server_status != "ACTIVE" ]]
do
   sleep 5
   server_status=$(openstack server list --format value -c Status --name "$SERVER_NAME")
done

if [ ! -z $server_status ] && [[ $server_status != "ERROR" ]]
then
   echo -n "$SERVER_NAME seems to be up and running. So we will add an external IP: "
   # Request a public floating IP (in the EXTERNAL_NETWORK_NAME network)
   # Q: Is there a way to keep track of where this IP is being used?
   server_ip=$(openstack floating ip create $EXTERNAL_NETWORK_NAME --format value -c floating_ip_address)
   echo "External IP: $server_ip"

   # Assign a public floating IP to my-first-server
   # NOTE: No point in doing the next step if you did not get server_ip
   openstack server add floating ip "$SERVER_NAME" "$server_ip"
fi

### Example command: get CPU usage

In [ ]:
# Run some commands on your remote server
cpu_usage=$(fgrep 'cpu ' /proc/stat | awk '{usage=($2+$4)*100/($2+$4+$5)} END {print usage}')
echo "Current CPU usage is $cpu_usage"

## Collect monitoring metrics

If you are using an image based on one of the Chameleon-provided base images (such as CC-CentOS7), your server will automatically register several monitoring metrics to a Gnocchi database. [Read the docs][1] to learn more about this.

There is a convenience Gnocchi client instance provided by the `chi` module. In fact, you can get an easy instance to most OpenStack clients this way.

[1]: https://chameleoncloud.readthedocs.io/en/latest/technical/metrics.html

In [ ]:
# Note: if your server just started, this might return no data. You should wait
# a bit for all the metrics to be registered in the system.
gnocchi = chi.gnocchi()
metric_name = 'load@load'
granularity = 60 # seconds
measures = gnocchi.metric.get_measures(metric_name, 
                                       resource_id=server.id, 
                                       granularity=granularity)

In [ ]:
# Plot using pandas
import pandas as pd
df = pd.DataFrame(data=measures, columns=('timestamp', 'granularity', 'value'))
df = df.set_index('timestamp').drop(columns='granularity')

df.plot()